In [1]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import statsmodels.stats.multitest as smm
from scipy import stats
import seaborn as sns

pd.options.mode.chained_assignment = None

#makes viewing pandas tables better
pd.set_option('display.max_colwidth', 0)

In [2]:
#CHANGE AS NEEDED

#Set this to the same as it was in BarSeq_counting.ipynb
output_dir = '/usr2/people/shollyt22/shollyt22/TnSeq_BarSeq_sequencingsBarseq_Temp_study_11_04_2023/TNSeq_mapping_output_new_genome_without_vector_seq_11_04_2023/TNSeq_barseq_outputs_11_06_2023'

### RHseq parameters


    ctrl_temp and exp_temp: Control and experimental temperatures, as written in the sample names for the metafile from BarSeq_counting.ipynb

    sp1, sp2: Species prefixes as in the TNseq annotation file (e.g. K. lactis is Kl) - must be the same length for both species

    include_promoters: If False, only analyze barcodes which landed in the CDS of a gene. If True, analyze barcodes which landed in the promoter and CDS.

    collapsed: 
    
        If collapsed = False, fitness ratios will be calculated separately for each experimental replicate: 
    
$$log_{2}(\frac{experimental\;replicate\;1\;normalized\;read\;count}{control\;average\;normalized\;read\;count});\; log_{2}(\frac{experimental\;replicate\;2\;normalized\;read\;count}{control\;average\;normalized\;read\;count}),\; etc.$$


        If collapsed = True, the fitness ratio will be caluclated from the average of experimental replicates: 
        
$$log_{2}\frac{experimental\;replicates\;AVERAGE\;normalized\;read\;count}{control\;average\;normalized\;read\;count}$$



    filter_strategy:

        If filter_strategy = 'coeff':
        Barcode will be kept if in at least one temperature, the coefficient of variation is LESS THAN than coeff_var_cutoff.

        If filter_strategy = 'reads':
        Barcode will be kept if in at least one temperature, the average normalized read count is GREATER THAN  norm_reads_cutoff.

        If filter_strategy = 'both':
        Barcode will be kept if it passes BOTH read count and coefficient of variation filters as described above.

    max_coeff_var_per_barcode: Maximum coefficient of variation of a barcode in a given temperature 
    
    min_norm_reads_per_barcode: Minimum average normalized read count for a barcode in a given temperature
    
    min_inserts_per_allele: Minimum number of barcodes which were inserted into each allele of a gene
    
    max_coeffvar_per_allele: Maximum coefficient of variation of fitness ratios for each allele of a gene

For reference, the following parameter set was used in the Saccharomyces Barseq paper:

    collapsed = False
    include_promoters = False

    filter_strategy = 'both'
    max_coeff_var_per_barcode = 2
    min_norm_reads_per_barcode = 1.1

    min_inserts_per_allele = 3
    max_coeffvar_per_allele = 10


In [4]:
#CHANGE AS NEEDED

ctrl_temp = 'OORB004_1'
exp_temp = 'OORB005_1'

sp1, sp2 = 'OORB004_1', 'OORB005_1'
prefix_length = len(sp1)

include_promoters = False
collapsed = False

filter_strategy = 'both'
max_coeff_var_per_barcode = 2
min_norm_reads_per_barcode = 1.1

min_inserts_per_allele = 3
max_coeffvar_per_allele = 10

### Load Barseq count data

In [5]:
df = pd.read_csv('{}/poolCount.txt'.format(output_dir), sep='\t')
df.head()

,barcode,scaffold,pos,gene,OORB004_1,OORB005_1
0,TTAACAGGTACTTTGAGCCT,chr4,1861183,proteinId=2306381,107,129
1,AGTGTGTTCTTCCAACCTAG,chr2,1265125,NaN,7413,7083
2,ATTAAAAGAGAATTAGGCTA,chr1,6115762,proteinId=2297294,32,31
3,TCATCATGGACCCGTGTAAT,chr2,4661375,proteinId=2302542,1791,2385
4,TGTCGGCCCGCAAAAACTCA,chr7,2325983,NaN,113,120


In [6]:
ctrl_cols = [col for col in df.columns if ctrl_temp in col]
exp_cols = [col for col in df.columns if exp_temp in col]
competition_cols = ctrl_cols + exp_cols

In [7]:
#check that these look correct
print('Control columns: {}'.format(ctrl_cols))
print('Experimental columns: {}'.format(exp_cols))

Control columns: ['OORB004_1']
Experimental columns: ['OORB005_1']


### Step 1: Normalize read counts

Normalized read count = (raw read count * average total mapped reads across replicates) / total mapped reads for replicate 

In [8]:
total_mapped_reads = {}

for col in competition_cols:
    total_mapped_reads[col] = sum(df[col])
    
avg_total_mapped_reads = np.mean(list(total_mapped_reads.values()))

for col in competition_cols:
    df[col+'_normalized_read_count'] = avg_total_mapped_reads*df[col]/total_mapped_reads[col]

In [9]:
df.head()

,barcode,scaffold,pos,gene,OORB004_1,OORB005_1,OORB004_1_normalized_read_count,OORB005_1_normalized_read_count
0,TTAACAGGTACTTTGAGCCT,chr4,1861183,proteinId=2306381,107,129,114.541364,121.031338
1,AGTGTGTTCTTCCAACCTAG,chr2,1265125,NaN,7413,7083,7935.468510,6645.464884
2,ATTAAAAGAGAATTAGGCTA,chr1,6115762,proteinId=2297294,32,31,34.255361,29.085050
3,TCATCATGGACCCGTGTAAT,chr2,4661375,proteinId=2302542,1791,2385,1917.229745,2237.672420
4,TGTCGGCCCGCAAAAACTCA,chr7,2325983,NaN,113,120,120.964244,112.587292


### Step 2: Filter barcodes

Drop barcodes with no reads in any Barseq competitions, barcodes in noncoding regions, and barcodes mapping to the plasmid.

In [10]:
print('Beginning with {} barcodes seen in the TNseq pool'.format(df.shape[0]))
print()

#drop barcodes with no reads in any competition
total_reads = df[competition_cols].sum(axis=1)
df = df[total_reads >= 1]
print('Dropping barcodes which were seen in the TNseq pool, but not in any Barseq competitions.')
print('{} barcodes remain for Barseq analysis.'.format(df.shape[0]))
print()

#drop barcodes with only 1 read across competitions
total_reads = df[competition_cols].sum(axis=1)
df = df[total_reads > 1]
print('Dropping barcodes with only 1 read across competitions.')
print('{} barcodes remain for Barseq analysis.'.format(df.shape[0]))
print()

#drop noncoding barcodes
df = df[~df['gene'].isna()]
print('Dropping barcodes in noncoding regions.')
print('{} barcodes remain for Barseq analysis.'.format(df.shape[0]))
print()


#drop barcodes mapping to plasmid
df = df[df['gene']!='plasmid']
print('Dropping barcodes mapping to the plasmid.')
print('{} barcodes remain for Barseq analysis.'.format(df.shape[0]))
print()

Beginning with 4986 barcodes seen in the TNseq pool

Dropping barcodes which were seen in the TNseq pool, but not in any Barseq competitions.
3707 barcodes remain for Barseq analysis.

Dropping barcodes with only 1 read across competitions.
3519 barcodes remain for Barseq analysis.

Dropping barcodes in noncoding regions.
1370 barcodes remain for Barseq analysis.

Dropping barcodes mapping to the plasmid.
1370 barcodes remain for Barseq analysis.



### Step 3: Calculate average of normalized read counts and coefficient of variation across replicates for each temperature.

In [11]:
#replace 0 with NaN so 0 values are not counted in calculating means

ctrl_norm_cols = [col + '_normalized_read_count' for col in ctrl_cols]
exp_norm_cols = [col + '_normalized_read_count' for col in exp_cols]

df[ctrl_norm_cols+exp_norm_cols] = df[ctrl_norm_cols+exp_norm_cols].replace(0.0, np.nan)

In [12]:
df[ctrl_temp+'_avg_normalized_read_count'] = df[ctrl_norm_cols].mean(axis=1)
df[ctrl_temp+'_coeffvar'] = df[ctrl_norm_cols].std(axis=1)/df[ctrl_temp+'_avg_normalized_read_count']

df[exp_temp+'_avg_normalized_read_count'] = df[exp_norm_cols].mean(axis=1)
df[exp_temp+'_coeffvar'] = df[exp_norm_cols].std(axis=1)/df[exp_temp+'_avg_normalized_read_count']


print('Mean coeff var for {} = {}.'.format(ctrl_temp, df[ctrl_temp+'_coeffvar'].mean()))
print('Mean coeff var for {} = {}.'.format(exp_temp, df[exp_temp+'_coeffvar'].mean()))

Mean coeff var for OORB004_1 = nan.
Mean coeff var for OORB005_1 = nan.


In [13]:
df.head()

,barcode,scaffold,pos,gene,OORB004_1,OORB005_1,OORB004_1_normalized_read_count,OORB005_1_normalized_read_count,OORB004_1_avg_normalized_read_count,OORB004_1_coeffvar,OORB005_1_avg_normalized_read_count,OORB005_1_coeffvar
0,TTAACAGGTACTTTGAGCCT,chr4,1861183,proteinId=2306381,107,129,114.541364,121.031338,114.541364,NaN,121.031338,NaN
2,ATTAAAAGAGAATTAGGCTA,chr1,6115762,proteinId=2297294,32,31,34.255361,29.085050,34.255361,NaN,29.085050,NaN
3,TCATCATGGACCCGTGTAAT,chr2,4661375,proteinId=2302542,1791,2385,1917.229745,2237.672420,1917.229745,NaN,2237.672420,NaN
5,CGAGTTCAAAGATGTTCAGT,chr3,4987853,proteinId=2305679,240,303,256.915209,284.282911,256.915209,NaN,284.282911,NaN
6,TTATCAGGCACGGACGAATT,chr1,547608,proteinId=2294147,395,454,422.839614,425.955253,422.839614,NaN,425.955253,NaN


### Step 4: Calculate fitness ratios


For EACH experimental replicate: fitness ratio = log2( experimental rep normalized read count / control average normalized read count)

Drop barcodes where the average fitness ratio is NaN.

In [14]:
exp_norm_cols = [col for col in df.columns if exp_temp in col and 'normalized_read_count' in col]

for col in exp_norm_cols:
    new_col = col.replace('normalized_read_count','fitness_ratio')
    df.loc[:, new_col] = np.log2(df[col]/df[ctrl_temp+'_avg_normalized_read_count'])

In [15]:
#drop barcodes where log2(exp_avg/ctrl_avg) is NaN
print('Dropping {} barcodes where log2(exp_avg/ctrl_avg) is NaN.'.format(df[df['{}_avg_fitness_ratio'.format(exp_temp)].isna()].shape[0]))
df = df[~df['{}_avg_fitness_ratio'.format(exp_temp)].isna()]
print('{} barcodes remain for Barseq analysis.'.format(df.shape[0]))

Dropping 70 barcodes where log2(exp_avg/ctrl_avg) is NaN.
1300 barcodes remain for Barseq analysis.


In [16]:
df.head()

,barcode,scaffold,pos,gene,OORB004_1,OORB005_1,OORB004_1_normalized_read_count,OORB005_1_normalized_read_count,OORB004_1_avg_normalized_read_count,OORB004_1_coeffvar,OORB005_1_avg_normalized_read_count,OORB005_1_coeffvar,OORB005_1_fitness_ratio,OORB005_1_avg_fitness_ratio
0,TTAACAGGTACTTTGAGCCT,chr4,1861183,proteinId=2306381,107,129,114.541364,121.031338,114.541364,NaN,121.031338,NaN,0.079512,0.079512
2,ATTAAAAGAGAATTAGGCTA,chr1,6115762,proteinId=2297294,32,31,34.255361,29.085050,34.255361,NaN,29.085050,NaN,-0.236052,-0.236052
3,TCATCATGGACCCGTGTAAT,chr2,4661375,proteinId=2302542,1791,2385,1917.229745,2237.672420,1917.229745,NaN,2237.672420,NaN,0.222976,0.222976
5,CGAGTTCAAAGATGTTCAGT,chr3,4987853,proteinId=2305679,240,303,256.915209,284.282911,256.915209,NaN,284.282911,NaN,0.146035,0.146035
6,TTATCAGGCACGGACGAATT,chr1,547608,proteinId=2294147,395,454,422.839614,425.955253,422.839614,NaN,425.955253,NaN,0.010591,0.010591


In [17]:
#save to csv
df.to_csv('{}/barcode_counts_and_fitness_ratios_UNFILTERED.csv'.format(output_dir))

### Step 4: Filter barcodes by coefficient of variation and/or average of normalized read counts.

In [18]:
coeff_var_cols = [exp_temp+'_coeffvar', ctrl_temp+'_coeffvar']
avg_norm_reads_cols = [exp_temp+'_avg_normalized_read_count', ctrl_temp+'_avg_normalized_read_count']

#make a copy for filtering so we don't mess up the original dataframe
df_copy = df.copy()

if filter_strategy == 'coeff':    
    filtered_df = df_copy[df_copy[coeff_var_cols].apply(lambda x: x <= max_coeff_var_per_barcode, axis=1).sum(axis=1) >= 1]
    print('Dropping {} barcodes with coeff >= {} in at least one temperature.'.format(df.shape[0]-filtered_df.shape[0], max_coeff_var_per_barcode))
    
elif filter_strategy == 'reads':
    filtered_df = df_copy[df_copy[avg_norm_reads_cols].apply(lambda x: x >= min_norm_reads_per_barcode, axis=1).sum(axis=1) >= 1]
    print('Dropping {} barcodes with average normalized read count <= {} in at least one temperature.'.format(df.shape[0]-filtered_df.shape[0], min_norm_reads_per_barcode))

elif filter_strategy == 'both':
    coeff_filt = df[df[coeff_var_cols].apply(lambda x: x <= max_coeff_var_per_barcode, axis=1).sum(axis=1) >= 1]
    read_filt = df[df[avg_norm_reads_cols].apply(lambda x: x >= min_norm_reads_per_barcode, axis=1).sum(axis=1) >= 1]
    filtered_df = df[(df.index.isin(coeff_filt.index)) & (df.index.isin(read_filt.index))]
    print('Dropping {} barcodes with coeff >= {} and/or average normalized read count <= {} in at least one temperature.'.format(df.shape[0]-filtered_df.shape[0], max_coeff_var_per_barcode, min_norm_reads_per_barcode))

print('{} barcodes remain for Barseq analysis.'.format(filtered_df.shape[0]))

Dropping 1300 barcodes with coeff >= 2 and/or average normalized read count <= 1.1 in at least one temperature.
0 barcodes remain for Barseq analysis.


In [19]:
output_file = '{}/barcode_counts_and_fitness_ratios_FILTERED.csv'.format(output_dir)

with open(output_file, 'w') as f:
    #write filter parameters to file
    f.write('# FILTER PARAMS: filter_strategy={}; max_coeff_var_per_barcode={}; min_norm_reads_per_barcode={}'.format(filter_strategy,max_coeff_var_per_barcode,min_norm_reads_per_barcode))
    #write remaining barcodes to file
    filtered_df.to_csv(f)

### Step 6: Filter by number of inserts in each gene/allele and coefficient of variation of inserts in each gene/allele.

Number of inserts means number of barcodes mapping to a given gene and allele.

Coefficient of variation of inserts means the coefficient of variation of 

Defaults from Melanie's scripts, change if needed.

In [21]:
#split annotation column into gene and allele columns
filtered_df['allele'] = filtered_df['gene'].str[:prefix_length]
filtered_df['gene_name'] = filtered_df['gene'].str[prefix_length:]

#gene names must be at least 3 characters long
filtered_df = filtered_df[filtered_df['gene_name'].str.len() >= 3]

In [22]:
#group barcodes by gene/allele and count number of inserts per allele
insert_counts_per_allele = filtered_df.groupby('gene_name')['allele'].value_counts().unstack(level=1)
insert_counts_per_allele.columns = [col+'_insert_counts' for col in insert_counts_per_allele.columns]

In [23]:
#group barcodes by gene/allele and calculate coefficient of variation of experimental/control average normalized read counts
def coeffvar(col):
    return np.std(col)/np.mean(col)

exp_ctrl_ratio = '{}/{}_avg_normalized_read_count'.format(exp_temp, ctrl_temp)
filtered_df[exp_ctrl_ratio] = filtered_df[exp_temp+'_avg_normalized_read_count']/filtered_df[ctrl_temp+'_avg_normalized_read_count']
coeff_var_by_allele = filtered_df.groupby(['gene_name', 'allele'])[exp_ctrl_ratio].apply(coeffvar).unstack(level=1)
coeff_var_by_allele.columns = [col+'_coeff_var' for col in coeff_var_by_allele.columns]

In [24]:
#merge inserts per allele and coeffvar data into one frame
inserts_per_allele = insert_counts_per_allele.merge(coeff_var_by_allele, left_index=True, right_index=True)
inserts_per_allele.head()

""
gene_name


In [25]:
print('Beginning with {} genes with at least one insertion in either species.'.format(inserts_per_allele.shape[0]))

inserts_per_allele_filtered_count = inserts_per_allele[(inserts_per_allele[sp1+'_insert_counts'] >= min_inserts_per_allele)
                                                 & (inserts_per_allele[sp2+'_insert_counts'] >= min_inserts_per_allele)]

print('Dropping {} genes with less than {} insertions per allele in one or both species.'.format(inserts_per_allele.shape[0]-inserts_per_allele_filtered_count.shape[0], min_inserts_per_allele))

inserts_per_allele_filtered = inserts_per_allele_filtered_count[(inserts_per_allele_filtered_count[sp1+'_coeff_var'] <= max_coeffvar_per_allele) 
                                                 & (inserts_per_allele_filtered_count[sp2+'_coeff_var'] <= max_coeffvar_per_allele)]

print('Dropping {} genes with coefficient of variation greater than {} in one or both species.'.format(inserts_per_allele_filtered_count.shape[0]-inserts_per_allele_filtered.shape[0], max_coeffvar_per_allele))


print('{} genes remain for analysis.'.format(inserts_per_allele_filtered.shape[0]))

Beginning with 0 genes with at least one insertion in either species.


KeyError: 'OORB004_1_insert_counts'

In [26]:
#drop barcodes mapping to genes that didn't pass above filter
filtered_df = filtered_df[filtered_df['gene_name'].isin(inserts_per_allele_filtered.index)]

NameError: name 'inserts_per_allele_filtered' is not defined

### Step 7: Calculate effect size, Mann Whitney U test and multiple hypothesis testing correction

Effect size = difference of fitness ratio means

In [ ]:
def multiple_columns_to_list(df):
    
    ###take values from multiple columns (eg. fitness ratios from different replicates) and 
    ###combine into a single list
    
    val_list = []
    for col in df.columns:
        val_list += df[col].dropna().tolist()
    return val_list

fitness_ratio_columns = [col for col in filtered_df.columns if 'fitness_ratio' in col and 'avg' not in col]
fitness_ratios_by_allele = filtered_df.groupby(['gene_name', 'allele'])[fitness_ratio_columns].apply(multiple_columns_to_list).to_frame()
fitness_ratios_by_allele = fitness_ratios_by_allele.unstack(level=1)
fitness_ratios_by_allele_columns = [col + '_fitness_ratios' for col in fitness_ratios_by_allele.columns.get_level_values(1)]
fitness_ratios_by_allele.columns = fitness_ratios_by_allele_columns
fitness_ratios_by_allele.head()

In [ ]:
##Effect size

fitness_ratios_by_allele[sp1+'_mean'] = fitness_ratios_by_allele[fitness_ratios_by_allele_columns[0]].apply(np.mean, axis=0)
fitness_ratios_by_allele[sp2+'_mean'] = fitness_ratios_by_allele[fitness_ratios_by_allele_columns[1]].apply(np.mean, axis=0)
fitness_ratios_by_allele['effect_size'] = fitness_ratios_by_allele[sp1+'_mean'] - fitness_ratios_by_allele[sp2+'_mean']

In [ ]:
##Mann Whitney U statistic
fitness_ratios_by_allele['MannWhitney_U'] = fitness_ratios_by_allele.apply(lambda x: 
                                     stats.mannwhitneyu(x[fitness_ratios_by_allele_columns[0]],
                                                        x[fitness_ratios_by_allele_columns[1]],
                                                       alternative='two-sided')[0], axis=1)
                                    
##Mann Whitney p value
fitness_ratios_by_allele['MannWhitney_p'] = fitness_ratios_by_allele.apply(lambda x: 
                                     stats.mannwhitneyu(x[fitness_ratios_by_allele_columns[0]],
                                                        x[fitness_ratios_by_allele_columns[1]],
                                                       alternative='two-sided')[1], axis=1)                            

In [ ]:
##Correction for multiple hypothesis testing
fitness_ratios_by_allele['MannWhitney_p_adj'] = smm.multipletests(fitness_ratios_by_allele['MannWhitney_p'].tolist(),
                                                                  method='fdr_bh')[1]

In [ ]:
##merge with insert count and coefficient of variation data above

fitness_ratios_by_allele = fitness_ratios_by_allele.merge(inserts_per_allele_filtered, left_index=True, right_index=True)
fitness_ratios_by_allele = fitness_ratios_by_allele.sort_values('MannWhitney_p_adj')
fitness_ratios_by_allele.head()

In [ ]:
#save to csv

output_file = '{}/allele_fitness_ratios_and_p_vals.csv'.format(output_dir)

with open(output_file, 'w') as f:
    #write filter parameters to file
    f.write('# FILTER PARAMS: filter_strategy={}; max_coeff_var_per_barcode={}; min_norm_reads_per_barcode={}; min_inserts_per_allele={}; max_coeffvar_per_allele={}'.format(filter_strategy,max_coeff_var_per_barcode,min_norm_reads_per_barcode,min_inserts_per_allele, max_coeffvar_per_allele))
    #write remaining barcodes to file
    fitness_ratios_by_allele.to_csv(f)

fitness_ratios_by_allele.to_csv('{}/allele_fitness_ratios_and_p_vals.csv'.format(output_dir))

### Anything with p <= 0.05?

In [ ]:
fitness_ratios_by_allele[fitness_ratios_by_allele['MannWhitney_p_adj'] <= 0.05]

### Plotting 

In [20]:
def plot_fitness_ratios_by_gene(gene):
    
    fitness_ratio_cols = [col for col in fitness_ratios_by_allele if '_fitness_ratios' in col]

    sns.kdeplot(fitness_ratios_by_allele.loc[gene][fitness_ratio_cols[0]], label=fitness_ratio_cols[0].split('_')[0])
    sns.kdeplot(fitness_ratios_by_allele.loc[gene][fitness_ratio_cols[1]], label=fitness_ratio_cols[1].split('_')[0])
    plt.legend()
    plt.title(gene)
    plt.xlabel('log2 {}/{}'.format(exp_temp, ctrl_temp))
    plt.show()

In [ ]:
plot_fitness_ratios_by_gene('YER022W')